<a href="https://colab.research.google.com/github/JaiVishwa55/Complete-Python-3-Bootcamp/blob/master/chatbotai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json
import random
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import json
from sklearn.svm import SVR

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
print(os.listdir('/content/drive'))


['MyDrive', '.shortcut-targets-by-id', '.Trash-0', '.Encrypted']


In [4]:
print(os.listdir('/content/drive/MyDrive'))


['Screenshot_20230208_001203.jpg', 'Jai Vishwa', 'Data Structures (1)', 'Data Structures', 'JAIVISHWA.pdf', 'IMG-20230730-WA0003.jpg', 'Fe results.gdoc', 'IMG_20240823_225236.jpg', 'Colab Notebooks', '17374547138438718285352634525581.jpg', 'Offer Letter - Jai.pdf', 'SIES_SDP_2025']


In [5]:
with open('/content/drive/MyDrive/intents.json','r') as file:
  data = json.load(file)
print(json.dumps(data, indent=4))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/intents.json'

In [6]:
print(os.listdir('/content/drive'))


['MyDrive', '.shortcut-targets-by-id', '.Trash-0', '.Encrypted']


In [7]:
import os
print(os.listdir('/content/drive/MyDrive'))


['Screenshot_20230208_001203.jpg', 'Jai Vishwa', 'Data Structures (1)', 'Data Structures', 'JAIVISHWA.pdf', 'IMG-20230730-WA0003.jpg', 'Fe results.gdoc', 'IMG_20240823_225236.jpg', 'Colab Notebooks', '17374547138438718285352634525581.jpg', 'Offer Letter - Jai.pdf', 'SIES_SDP_2025']


In [8]:
file_path = '/content/drive/MyDrive/intents.json'


In [9]:
import json

file_path = '/content/drive/MyDrive/intents.json'

with open(file_path, 'r') as file:
    data = json.load(file)

print(json.dumps(data, indent=4))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/intents.json'

In [10]:
tagCount = {}
for intent in data['intents']:
  tag = intent['tag']
  if tag not in tagCount:
    tagCount[tag] = 0
  tagCount[tag] += len(intent['patterns'])
print(tagCount)


NameError: name 'data' is not defined

In [11]:
question_lengths = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        question_lengths.append(len(pattern))

print(f"Minimum question length: {min(question_lengths)}")
print(f"Maximum question length: {max(question_lengths)}")
print(f"Average question length: {sum(question_lengths) / len(question_lengths):.2f}")

question_word_counts = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        question_word_counts.append(len(pattern.split()))

print(f"\nMinimum question word count: {min(question_word_counts)}")
print(f"Maximum question word count: {max(question_word_counts)}")
print(f"Average question word count: {sum(question_word_counts) / len(question_word_counts):.2f}")

import matplotlib.pyplot as plt
plt.hist(question_lengths, bins=20)
plt.xlabel("Question Length (Characters)")
plt.ylabel("Frequency")
plt.title("Distribution of Question Lengths")
plt.show()

plt.hist(question_word_counts, bins=20)
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.title("Word count distribution")
plt.show()

NameError: name 'data' is not defined

In [ ]:
answer_lengths = []
for intent in data['intents']:
    answer_lengths.append(len(intent['responses'][0]))  # Character length

print(f"Minimum answer length: {min(answer_lengths)}")
print(f"Maximum answer length: {max(answer_lengths)}")
print(f"Average answer length: {sum(answer_lengths) / len(answer_lengths):.2f}")

# Word counts instead of character counts
answers_word_counts = []
for intent in data['intents']:
    answers_word_counts.append(len(intent['responses'][0].split()))

print(f"\nMinimum answers word count: {min(answers_word_counts)}")
print(f"Maximum answers word count: {max(answers_word_counts)}")
print(f"Average answers word count: {sum(answers_word_counts) / len(answers_word_counts):.2f}")

plt.hist(answer_lengths, bins=20)  # Adjust bins as needed
plt.xlabel("Answer Length (Characters)")
plt.ylabel("Frequency")
plt.title("Distribution of Answer Lengths")
plt.show()

plt.hist(answers_word_counts, bins=20)
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.title("Word count distribution")
plt.show()

In [12]:
from sklearn.ensemble import RandomForestClassifier


nltk.download('punkt')

questions = []
answers = []
tags = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        questions.append(pattern)
        answers.append(intent['responses'][0])
        tags.append(intent['tag'])

df = pd.DataFrame({'Question': questions, 'Answer': answers, 'Tag': tags})

def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(f"[{string.punctuation}]", "", text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

df['Question'] = df['Question'].apply(clean_text)

X = df['Question']
y = df['Tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_estimators=100))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


def get_response(user_input):
    cleaned_input = clean_text(user_input)
    predicted_tag = model.predict([cleaned_input])[0]
    for intent in data['intents']:
        if intent['tag'] == predicted_tag:
            return random.choice(intent['responses'])
    return "I'm sorry, I don't understand that."

def run_chatbot():
    print("Welcome to the NLP Chatbot! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = get_response(user_input)
        print(f"Chatbot: {response}")

run_chatbot()


NameError: name 'nltk' is not defined